### 4D Poisson Equation
Let $\Omega = (-1 , 1)^4, \omega \in \mathbb{R}$:

$$
\begin{cases}
    \Delta u + f = 0, (w,x,y,z) \in \Omega\\
    u = g, (w,x,y,z) \in \partial \Omega
\end{cases}
$$

with:
$$
    f(x,y) =  -4 \omega^2 g\\
$$

### Ground Truth
$$
g(x,y) = \sin(\omega w) \cos(\omega x) \sin(\omega y) \cos(\omega z)
$$

In [2]:
import numpy as np
import torch
import scipy
import time
import sys
sys.path.insert(1, './PSM_V2')
from sobolev import Sobolev
from solver import Solver
from utils import matmul, cart
from diffeomorphism import hyper_rect
import surrogates

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float64)

### Sobolev Cubature

In [31]:
deg_4d = [8,8,8,8]
deg_3d = [8]*3

ints = np.array([[-1.0, 1.0]]*4)
diffeo = hyper_rect(*ints)
phi, _ = diffeo

sob_4d = Sobolev(deg=deg_4d)
sob_3d = Sobolev(deg=deg_3d)

diffs = np.array([[2,0,0,0], [0,2,0,0], [0,0,2,0],[0,0,0,2]])

In [32]:
dw2, dx2, dy2, dz2 = torch.tensor(sob_4d.diff.diffs(diffs))

### Surrogate Model

In [33]:
model = surrogates.ChebPoly(n=deg_4d, p=2, dim=4)
print(f'params: {model.get_deg()}')

params: 1867


### Tests

In [34]:
# {omega = 1.0}

### Data

In [35]:
omega = 1.0

def gt(w,x,y,z):
    v = phi([w,x,y,z])
    return np.cos(omega*v[0])*np.sin(omega*v[1])*np.cos(omega*v[2])*np.sin(omega*v[3])

def f(w,x,y,z):
    return 4*omega**2*gt(w,x,y,z)

### Sobolev Order

In [36]:
sob_4d.set_s(-1)

### Sobolev Metric

In [37]:
# |--------------------------------------------|
# |  Operator  |          Formulation          |  
# |------------|-------------------------------|
# | id         |  L2 grad of L2                | 
# | m_inv      |  L2 grad of Sob               |
# | weak m_inv |  L2 grad of weak Sob          |
# | m          |  L2 grad of negative Sob      |
# | weak m     |  L2 grad of weak negative Sob |
# |--------------------------------------------|
#
# -> sob.metric(rev=False/True, weak=False/True)

In [38]:
metric_4d = sob_4d.metric(weak=True)
metric_3d = sob_3d.l2_metric()

### Gradient Flow :-: Model Input

In [39]:
grid = sob_4d.leja_grid
xs = sob_4d.leja_axes

bndr = np.array([-1.0, 1.0])
xs_bndr = sob_3d.leja_axes

dmn = model.data_axes(xs).T

bndr_prot = np.array([bndr, *xs_bndr])

bndr = model.data_axes(bndr_prot).T
for i in range(1, 4):
    bndr = torch.cat((model.data_axes(np.roll(bndr_prot, i)).T, bndr))

/tmp/ipykernel_29630/203345870.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bndr_prot = np.array([bndr, *xs_bndr])


### Gradient Flow :-: Data Input

In [40]:
grid_bndr = cart(bndr_prot)
for i in range(1, 4):
    grid_bndr = np.concatenate((cart(np.roll(bndr_prot, i)), grid_bndr))

u_bndr = torch.tensor(gt(
    grid_bndr[:,0], grid_bndr[:,1], grid_bndr[:,2], grid_bndr[:,3], 
))

fWXYZ = torch.tensor(f(grid[:,0], grid[:,1], grid[:,2], grid[:,3]))

### PDE :-: Operators

In [41]:
K = dw2+dx2+dy2+dz2

### Gradient Flow :-: Formulation

In [42]:
eq = lambda u: matmul(K, u)+fWXYZ
crit_dmn = lambda u: sob_4d.loss(eq(u), weak=True)
crit_bndr = lambda u: sob_3d.l2_loss(u-u_bndr)
grad_dmn = lambda u: 2*matmul(K.T, metric_4d(eq(u)))
grad_bndr = lambda u: 2*metric_3d(u-u_bndr)

### Truncation Error

In [43]:
crit_dmn(torch.tensor(
    gt(grid[:,0], grid[:,1], grid[:,2], grid[:,3])
))

tensor(1.5239e-13)

### Solver

In [44]:
test_xs = [np.linspace(-1.0, 1.0, 20)]*4
data = model.data_axes(test_xs).T

solver = Solver(
    dmns=[dmn, bndr],
    crits=[crit_dmn, crit_bndr],
    model=model,
    test_axes=test_xs,
    grads=[grad_dmn, grad_bndr],
    gt=gt,
    data=data)

### Resolution

In [45]:
grid_plt = sob_4d.grid
xs_plt = sob_4d.axes
w_plt = grid_plt[:,0]
x_plt = grid_plt[:,1]
y_plt = grid_plt[:,2]
z_plt = grid_plt[:,3]
gt_plt = gt(
    w_plt, x_plt, y_plt, z_plt
).reshape(len(xs_plt[3]),len(xs_plt[2]),len(xs_plt[1]),len(xs_plt[0]))
fn = None #"resolution_poisson_2d"

#solver.plot4d(gt_plt, xs_plt[0], xs_plt[1], title="Resolution of Ground Truth", file_name=fn)

### Remark
You can either decide for the analytic solution or for the iterative solution.

### Gradient Flow :-: Analytic Solution AD-PSM

In [46]:
start = time.time()
KsK = 2*matmul(dmn.T, K.T, metric_4d(matmul(K, dmn)))\
        +2*matmul(bndr.T, metric_3d(bndr))

Ksf = 2*matmul(dmn.T, K.T, metric_4d(-fWXYZ))\
        +2*matmul(bndr.T, metric_3d(u_bndr))

w = matmul(KsK.inverse(), Ksf)
model.set_weights(w)
end = time.time()
print('time consumption: %.2fs' % (end-start))

time consumption: 2.43s


## Errors AD-PSM

In [50]:
_, _ = solver.eval()

In [51]:
print(f'L1  Error: {solver.lp_err(1)}')
print(f'L2  Error: {solver.lp_err(2)}')
print(f'Max Error: {solver.lp_err(np.inf)}')

L1  Error: 5.4172772194542376e-08
L2  Error: 8.741464462050605e-08
Max Error: 6.37169315975683e-07


### Gradient Flow :-: Iterative Solution

In [52]:
model.set_weights_val(0.0)

In [53]:
params = model.parameters()
optimizer = torch.optim.LBFGS(
    params,
    lr=1.0, 
    max_iter=1, 
    max_eval=None, 
    tolerance_grad=1e-18, 
    tolerance_change=1e-20, 
    history_size=1, 
    line_search_fn=None)
solver.train(200, 20, optim=optimizer)
print('time consumption: %.2fs' % solver.get_time())

epoch 20: loss = 0.2318489257961869
epoch 40: loss = 0.0107052822668528
epoch 60: loss = 0.0009490983598119
epoch 80: loss = 0.0002270859135860
epoch 100: loss = 0.0000196032587589
epoch 120: loss = 0.0000064368497951
epoch 140: loss = 0.0000025047524065
epoch 160: loss = 0.0000005463266355
epoch 180: loss = 0.0000001379899735
epoch 200: loss = 0.0000001244969927
time consumption: 11.37s


### Evaluate

In [54]:
_, _ = solver.eval()

### Plots

In [55]:
suffix = None#"poisson2d"
#solver.plot_model(suffix=suffix)
#solver.plot_gt(suffix=suffix)
#solver.plot_abs_err(suffix=suffix)
#solver.plot_losses(lower=0, upper=-1)

### Errors GF-PSM

In [57]:
print(f'L1  Error: {solver.lp_err(1)}')
print(f'L2  Error: {solver.lp_err(2)}')
print(f'Max Error: {solver.lp_err(np.inf)}')

L1  Error: 1.406531625406868e-05
L2  Error: 3.580693815416279e-05
Max Error: 0.0032409841233420333
